In [15]:
import numpy as np
import cv2, os
import matplotlib.pyplot as plt
import dlib
from imutils.face_utils import FaceAligner

In [16]:
"""

1. pose_predictor is used to get the 68 facial landmarks detection.
2. face_encoder is used to get the (128,1) dimension encoding of the image which is passed to it.
   It is a pretrained ResNet network model with 29 conv layers. The model is trained on a dataset of about 3 million faces.
3. detector is an object of dlib.get_frontal_face_detector() which is used to get the front face from the face image.
4. modelFile, configFile are the files for the dnn based cv2 face detection model.
   It is a quantized tensorflow model which is based on the Single Shot-Multibox Detector (SSD) and uses ResNet-10 architecture as its backbone. It was introduced post OpenCV 3.3 in its deep neural network module.
5. net is the object of the cv2.dnn module that is initialized with the model and config file.

"""

pose_predictor=dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

fa = FaceAligner(pose_predictor)

face_encoder=dlib.face_recognition_model_v1('dlib_face_recognition_resnet_model_v1.dat')

detector = dlib.get_frontal_face_detector()

modelFile = 'opencv_face_detector_uint8.pb'

configFile = 'opencv_face_detector.pbtxt'

net = cv2.dnn.readNetFromTensorflow(modelFile, configFile)


In [45]:
faces=[]
name=[]


In [54]:
for im in os.listdir(trainpath):
    print(im)
    img = cv2.imread(os.path.join(trainpath,im))
   
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    frameHeight = img.shape[0]
    frameWidth = img.shape[1]
    blob = cv2.dnn.blobFromImage(img, 1.0, (300, 300), [104, 117,   123], False, False)
    net.setInput(blob)
    detections = net.forward()
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.7:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            faceAligned =fa.align(img, gray,dlib.rectangle(x1,y1,x2,y2))
            landmark = pose_predictor(faceAligned,dlib.rectangle(0,0,faceAligned.shape[0],faceAligned.shape[1]))
            face_descriptor = face_encoder.compute_face_descriptor(faceAligned, landmark, num_jitters=2)
            faces.append(face_descriptor)
            im = im.split('_')
            name.append(im[0]+"_"+im[1])

AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [47]:
faces = np.array(faces)
name = np.array(name)
np.save('face_repr.npy', faces)
np.save('labels.npy', name)

In [48]:
faces = np.load("face_repr.npy")
name = np.load("labels.npy")

In [50]:
##Testing
testpath = "./Ajaytest"
for im in os.listdir(testpath):
    
    image = cv2.imread(os.path.join(testpath,im))
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    frameHeight = image.shape[0]
    frameWidth = image.shape[1]
    blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), [104, 117, 123], False, False)
    net.setInput(blob)
    detections = net.forward()

    scores=[]

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.7:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            faceAligned = fa.align(image, gray,dlib.rectangle(x1,y1,x2,y2))
            landmark = pose_predictor(faceAligned,dlib.rectangle(0,0,faceAligned.shape[0],faceAligned.shape[1]))
            face_descriptor = face_encoder.compute_face_descriptor(faceAligned, landmark, num_jitters=2)
            score = np.linalg.norm(faces - np.array(face_descriptor), axis=1)
            scores.append(score)
            imatches = np.argsort(score)
            score = score[imatches]
            print(im)
            print(name[imatches][:10].tolist(), score[:10].tolist())

Ajay.jpg
['ajay_kumar.jpg', 'Antonio_Banderas', 'Ben_Kingsley', 'Adrien_Brody', 'Brad_Garrett', 'Andy_Serkis', 'Alfred_Molina', 'Brendan_Fraser', 'Bradley_Cooper', 'Alan_Alda'] [0.2749846364781512, 0.7356605475858266, 0.7542944954965313, 0.7636067829359392, 0.7641578906067678, 0.7646803863877131, 0.7667155713513225, 0.7870571949613655, 0.7940957565941018, 0.800250047632669]
ajay_kumar.jpg
['ajay_kumar.jpg', 'Antonio_Banderas', 'Andy_Serkis', 'Ben_Kingsley', 'Alec_Baldwin', 'Brad_Garrett', 'Brendan_Fraser', 'Adam_Brody', 'Ashton_Kutcher', 'Bradley_Cooper'] [0.11046305156011715, 0.714731187686019, 0.7280857876385486, 0.7377160991009825, 0.7537254712458972, 0.7564561019579998, 0.7660290501885487, 0.7671774818053045, 0.7699488787785754, 0.7815716208744328]
